The dataset used in this notebook can be found at:
https://www.kaggle.com/datasets/samithsachidanandan/human-face-emotions

In [ ]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torch
import torch.optim as optim
from torch.utils.data import Subset
from skorch import NeuralNetClassifier
from skorch.callbacks import EarlyStopping

import seaborn as sns
from itertools import islice
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from pathlib import Path
import mlflow
import mlflow.pytorch

from utils.helper import find_project_root
from utils.mlflow import is_mlflow_server_running, set_mlflow_tracking_uri, log_run

In [ ]:
IMG_SIZE = 64
DATASET_PATH = find_project_root() / Path("datasets/computer_vision/Emotions/")

FIGURES_DIR = Path("figures/emotions/")
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

if not is_mlflow_server_running():
    raise RuntimeError("MLflow server is not running. Please start the MLflow server before running this notebook.")

set_mlflow_tracking_uri()
mlflow.set_experiment("Emotions_Classification")

In [ ]:
from computer_vision.data.transforms import get_transform

transform = get_transform(IMG_SIZE, [0.5117, 0.5098, 0.5089], [0.2070, 0.2062, 0.2060])

# the above mean and std were computed from the training set with the following function:
from computer_vision.data.transforms import compute_normalization

ds = ImageFolder(root=DATASET_PATH,
                 transform=transform)

In [ ]:
TEST_SIZE = 0.2
SEED = 42

# generate indices: instead of the actual data we pass in integers instead
train_indices, test_indices, _, _ = train_test_split(
    range(len(ds)),
    ds.targets,
    stratify=ds.targets,
    test_size=TEST_SIZE,
    random_state=SEED
)

# generate subset based on indices
train_dataset = Subset(ds, train_indices)
test_dataset = Subset(ds, test_indices)

In [ ]:
len(test_dataset), len(train_dataset)

In [ ]:
y_train = np.array([y for x, y in iter(train_dataset)])
y_test = np.array([y for x, y in iter(test_dataset)])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Visualize some images

In [ ]:
X_example, y_example = zip(*islice(iter(train_dataset), 7))

In [ ]:
from computer_vision.utils.figures import plot_example

plot_example(torch.stack(X_example), y_example, ds.classes, n=7, mean=transform.transforms[2].mean, std=transform.transforms[2].std);

In [ ]:
# Are images RGB or grayscale?
print(f"Image shape: {X_example[0].shape}")  # should print (3, IMG_SIZE, IMG_SIZE) for RGB images

# Distribution of classes in the training set

In [ ]:
from computer_vision.utils.figures import plot_label_distribution

figures_name = FIGURES_DIR / "class_distribution.png"

plot_label_distribution(ds, figures_name)

# Training a baseline model

In [ ]:
from computer_vision.models.baseline import BaselineModel
torch.manual_seed(0)

params = {
    'max_epochs': 10,
    'lr': 0.01,
    'batch_size': 64,
}

baseline = NeuralNetClassifier(
    BaselineModel,
    iterator_train__num_workers=2,
    iterator_valid__num_workers=2,
    callbacks=[EarlyStopping(patience=3)],
    device=device,
    module__input_dim=IMG_SIZE*IMG_SIZE*3,
    **params
)

In [ ]:
log_run(baseline, train_dataset, y_train, test_dataset, y_test, run_name="baseline_run", params=params)

In [ ]:
train_loss_history = baseline.history[:, 'train_loss']
valid_loss_history = baseline.history[:, 'valid_loss']
plt.figure()
sns.lineplot(x=range(1, len(train_loss_history) + 1), y=train_loss_history, label='Train Loss')
sns.lineplot(x=range(1, len(valid_loss_history) + 1), y=valid_loss_history , label='Validation Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("BaseCNN: Training Loss over Epochs")
cnn_fig = FIGURES_DIR / "cnn_loss.png"
plt.savefig(cnn_fig)
plt.show()

# Use BaseCNN with custom parameters

In [ ]:
from computer_vision.models.BaseCNN import BaseCNN
torch.manual_seed(0)

params = {
    'max_epochs': 10,
    'lr': 0.001,
    'optimizer': optim.Adam,
    'batch_size': 64,
    'callbacks': [EarlyStopping(patience=3)],

    'module__img_size': IMG_SIZE,
    'module__nb_conv_layers': 2,
    'module__nb_layers': 2,
    'module__net_width': 256,
    'module__dropout_rates': [0.25, 0.5],
}

cnn = NeuralNetClassifier(
    BaseCNN,
    #max_epochs=10,
    #lr=0.001,
    #optimizer=optim.Adam,
    device=device,
    #callbacks=[EarlyStopping(patience=3)],

    module__num_classes=5,
    #module__nb_img_channels=1, # grayscale images

    **params
)

In [ ]:
log_run(cnn, train_dataset, y_train, test_dataset, y_test, run_name="basecnn_run", params=params)

In [ ]:
train_loss_history = cnn.history[:, 'train_loss']
valid_loss_history = cnn.history[:, 'valid_loss']
plt.figure()
sns.lineplot(x=range(1, len(train_loss_history) + 1), y=train_loss_history, label='Train Loss')
sns.lineplot(x=range(1, len(valid_loss_history) + 1), y=valid_loss_history , label='Validation Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("BaseCNN: Training Loss over Epochs")
cnn_fig = FIGURES_DIR / "cnn_loss.png"
plt.savefig(cnn_fig)
plt.show()

# Hyperparameter Tuning with Grid Search

In [ ]:
from computer_vision.models.BaseCNN import BaseCNN
from sklearn.model_selection import GridSearchCV
from skorch.helper import SliceDataset

net = NeuralNetClassifier(
    BaseCNN,
    max_epochs=10,
    lr=0.01,
    batch_size=64,
    callbacks=[EarlyStopping(patience=3)],

    module__num_classes=5,
    module__img_size=IMG_SIZE,
    #module__nb_img_channels=1, # grayscale images
    device=device,

)

In [ ]:
params = [
    {
        'optimizer': [optim.Adam],
        'lr': [1e-3],
        'module__nb_conv_layers': [2, 3],
        'module__nb_layers': [2, 3],
        'module__net_width': [128, 256],
    },
    {
        'optimizer': [optim.SGD],
        'lr': [0.05],
        'module__nb_conv_layers': [2, 3],
        'module__nb_layers': [2, 3],
        'module__net_width': [128, 256],
    }
]

In [ ]:
grid = GridSearchCV(net, params, cv=2, scoring='accuracy', verbose=2, n_jobs=-1)

In [ ]:
train_dataset_sliceable = SliceDataset(train_dataset)
test_dataset_sliceable = SliceDataset(test_dataset)

In [ ]:
log_run(grid, train_dataset_sliceable, y_train, test_dataset_sliceable, y_test, run_name="gridsearch_run", params=params)